In [ ]:
#DON'T RENAME ME - I'M CHANGED EXTERNALLY
MSNAME = "msdir/COMBINED.ms"
UNPOL_SOURCE = "PKS1934-638"

In [ ]:
from IPython.display import HTML
HTML('code_toggle.html')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from pyrap.tables import table as tbl
from pyrap.tables import taql as taql
from scipy.stats import kurtosis, skew
%matplotlib inline

In [ ]:
with tbl("%s::FIELD" % MSNAME) as t:
    fnames = list(t.getcol("NAME"))
    print fnames
with tbl("%s::SPECTRAL_WINDOW" % MSNAME) as t:
    chans = t.getcell("CHAN_FREQ", 0).size
with tbl("%s" % MSNAME) as t:
    nrow = t.nrows()

with tbl("%s" % MSNAME, ack=False) as t:
    upol_source_id = fnames.index(UNPOL_SOURCE)
    with taql('select from $t where FIELD_ID == $upol_source_id') as t1:
        a1 = t1.getcol("ANTENNA1")
        a2 = t1.getcol("ANTENNA2")
        flgs = t1.getcol("FLAG")
        data = t1.getcol("CORRECTED_DATA")

In [ ]:
fsel = np.sum(flgs, axis=2) > 0
I = 0.5 * (data[:,:,0] + data[:,:,3])  
Q = 0.5 * (data[:,:,0] - data[:,:,3])   
U = 0.5 * (data[:,:,1] + data[:,:,2])  
V = 0.5 * (data[:,:,1] - data[:,:,2]) * 1.0j 
# flag data
I[fsel] = np.nan
Q[fsel] = np.nan
U[fsel] = np.nan
V[fsel] = np.nan

In [ ]:
lin_leakage = np.abs((Q**2 + U**2) / I**2)
total_leakage = np.abs((Q**2 + U**2 + V**2) / I**2)

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(12,4))
ax1.scatter(V[:,:],U[:,:])
ax1.set_xlabel("Stokes V")
ax1.set_ylabel("Stokes U")
ax2.scatter(V[:,:],Q[:,:])
ax2.set_xlabel("Stokes V")
ax2.set_ylabel("Stokes Q")
ax3.scatter(U[:,:],Q[:,:])
ax3.set_xlabel("Stokes U")
ax3.set_ylabel("Stokes Q")
plt.show()

In [ ]:
print "Spread in V:", np.nanmean(np.abs(V[:,:])), "+/-", np.nanstd(np.abs(V[:,:])), "kurt:", kurtosis(np.abs(V.ravel()), nan_policy='omit'), "skew:", skew(np.abs(V[np.logical_not(np.isnan(V))]).ravel()), "Jy"
print "Spread in U:", np.nanmean(np.abs(U[:,:])), "+/-", np.nanstd(np.abs(U[:,:])), "kurt:", kurtosis(np.abs(U.ravel()), nan_policy='omit'), "skew:", skew(np.abs(U[np.logical_not(np.isnan(U))]).ravel()), "Jy"
print "Spread in Q:", np.nanmean(np.abs(Q[:,:])), "+/-", np.nanstd(np.abs(Q[:,:])), "kurt:", kurtosis(np.abs(Q.ravel()), nan_policy='omit'), "skew:", skew(np.abs(Q[np.logical_not(np.isnan(Q))]).ravel()), "Jy"

In [ ]:
lin_leakage[lin_leakage == 0] = np.nan
total_leakage[lin_leakage == 0] = np.nan
print "Mean linear leakage:", np.nanmean(lin_leakage), \
      "+/-", np.nanstd(lin_leakage), \
      "(", 10*np.log10(np.nanmean(lin_leakage) + np.nanstd(lin_leakage)), "~", \
      10*np.log10(np.nanmean(lin_leakage)), "dB)"
print "Mean total leakage:", np.nanmean(total_leakage), \
      "+/-", np.nanstd(total_leakage), \
      "(", 10*np.log10(np.nanmean(total_leakage) + np.nanstd(total_leakage)), "~", \
      10*np.log10(np.nanmean(total_leakage)), "dB)"

In [ ]:
plt.figure(figsize=(12,6))
f = np.linspace(856,856*2,chans)
plt.errorbar(f,
             np.nanmean(Q, axis=0),
             yerr=np.nanstd(Q, axis=0), fmt='o', ecolor='r', capsize=4)
plt.plot(f, np.nanmedian(Q, axis=0), 'b--')
plt.xlabel("Frequency (MHz)")
plt.ylabel("Stokes Q")
plt.xlim(900,1600)
plt.show()
plt.figure(figsize=(12,6))
f = np.linspace(856,856*2,chans)
plt.errorbar(f,
             np.nanmean(U, axis=0),
             yerr=np.nanstd(U, axis=0), fmt='o', ecolor='r', capsize=4)
plt.plot(f, np.nanmedian(U, axis=0), 'b--')
plt.xlabel("Frequency (MHz)")
plt.ylabel("Stokes U")
plt.xlim(900,1600)
plt.show()
plt.figure(figsize=(12,6))
f = np.linspace(856,856*2,chans)
f = np.linspace(856,856*2,chans)
plt.errorbar(f,
             np.nanmean(V, axis=0),
             yerr=np.nanstd(V, axis=0), fmt='o', ecolor='r', capsize=4)
plt.plot(f, np.nanmedian(V, axis=0), 'b--')
plt.xlabel("Frequency (MHz)")
plt.ylabel("Stokes V (calibration error)")
plt.xlim(900,1600)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
fleak_mean = np.nanmean(lin_leakage, axis=0)
fleak_std = np.nanstd(lin_leakage, axis=0)
f = np.linspace(856,856*2,chans)
plt.fill_between(f,
                 10*np.log10(np.abs(fleak_mean)),
                 10*np.log10(np.abs(fleak_mean + fleak_std)))
plt.xlabel("Frequency (MHz)")
plt.ylabel("$D_{lin}$ (dB)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
fleak_mean_tot = np.nanmean(total_leakage, axis=0)
fleak_std_tot = np.nanstd(total_leakage, axis=0)
f = np.linspace(856,856*2,chans)
plt.fill_between(f,
                 10*np.log10(np.abs(fleak_mean_tot)),
                 10*np.log10(np.abs(fleak_mean_tot + fleak_std_tot)))
plt.xlabel("Frequency (MHz)")
plt.ylabel("$D_{tot}$ (dB)")
plt.grid(True)
plt.show()